In [1]:
import pandas as pd

In [2]:
imdb = pd.read_csv('./IMDB-Movie-Data.csv')

In [26]:
imdb['Description'][1]

'Following clues to the origin of mankind, a team finds a structure on a distant moon, but they soon realize they are not alone.'

In [4]:
imdb.describe().T

,count,mean,std,min,25%,50%,75%,max
Rank,1000.0,500.500000,288.819436,1.0,250.75,500.500,750.250,1000.00
Year,1000.0,2012.783000,3.205962,2006.0,2010.00,2014.000,2016.000,2016.00
Runtime (Minutes),1000.0,113.172000,18.810908,66.0,100.00,111.000,123.000,191.00
Rating,1000.0,6.723200,0.945429,1.9,6.20,6.800,7.400,9.00
Votes,1000.0,169808.255000,188762.647518,61.0,36309.00,110799.000,239909.750,1791916.00
Revenue (Millions),872.0,82.956376,103.253540,0.0,13.27,47.985,113.715,936.63
Metascore,936.0,58.985043,17.194757,11.0,47.00,59.500,72.000,100.00


In [5]:
imdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
Rank                  1000 non-null int64
Title                 1000 non-null object
Genre                 1000 non-null object
Description           1000 non-null object
Director              1000 non-null object
Actors                1000 non-null object
Year                  1000 non-null int64
Runtime (Minutes)     1000 non-null int64
Rating                1000 non-null float64
Votes                 1000 non-null int64
Revenue (Millions)    872 non-null float64
Metascore             936 non-null float64
dtypes: float64(3), int64(4), object(5)
memory usage: 93.8+ KB


In [6]:
C=imdb['Rating'].mean()

In [7]:
m = imdb['Votes'].quantile(0.9)

In [8]:
q_moives = imdb.copy().loc[imdb['Votes']>=m]
q_moives.shape

(100, 12)

In [9]:
# 영화 평점 기준 함수
def WR(x, m=m, C=C):
    v = x['Votes']
    R = x['Rating']
    return (v / (v+m)*R) + (m / (m+v)*C)

In [10]:
# 함수 적용할 때는 apply
imdb['Score'] = imdb.apply(WR, axis=1)

In [12]:
imdb['Rating']

0      8.1
1      7.0
2      7.3
3      7.2
4      6.2
5      6.1
6      8.3
7      6.4
8      7.1
9      7.0
10     7.5
11     7.8
12     7.9
13     7.7
14     6.4
15     6.6
16     8.2
17     6.7
18     8.1
19     8.0
20     6.7
21     7.9
22     6.7
23     6.5
24     5.3
25     6.8
26     8.3
27     4.7
28     6.2
29     5.9
      ... 
970    4.8
971    6.9
972    5.9
973    6.2
974    7.4
975    6.0
976    6.2
977    5.0
978    7.0
979    7.6
980    7.0
981    5.3
982    7.4
983    6.5
984    6.8
985    5.6
986    5.9
987    6.3
988    7.1
989    7.5
990    6.6
991    8.5
992    6.3
993    5.9
994    6.7
995    6.2
996    5.5
997    6.2
998    5.6
999    5.3
Name: Rating, Length: 1000, dtype: float64

In [13]:
imdb.columns

Index(['Rank', 'Title', 'Genre', 'Description', 'Director', 'Actors', 'Year',
       'Runtime (Minutes)', 'Rating', 'Votes', 'Revenue (Millions)',
       'Metascore', 'Score'],
      dtype='object')

In [14]:
col = ['Title', 'Rank' , 'Votes', 'Score']
rs_original = imdb.sort_values('Rating', ascending=False)[col]
rs_new = imdb.sort_values('Score', ascending=False)[col]

In [16]:
rs_new.head()

,Title,Rank,Votes,Score
54,The Dark Knight,55,1791916,8.579216
80,Inception,81,1583625,8.376003
36,Interstellar,37,1047747,8.075617
124,The Dark Knight Rises,125,1222645,8.056861
99,The Departed,100,937414,7.962793


In [17]:
imdb['Description'].head()

0    A group of intergalactic criminals are forced ...
1    Following clues to the origin of mankind, a te...
2    Three girls are kidnapped by a man with a diag...
3    In a city of humanoid animals, a hustling thea...
4    A secret government agency recruits some of th...
Name: Description, dtype: object

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
#TF-IDF는 정보 검색과 텍스트 마이닝에서 이용하는 가중치로, 
#여러 문서로 이루어진 문서군이 있을 때 어떤 단어가 특정 문서 내에서 얼마나 중요한 것인지를 나타내는 통계적 수치이다. 
#문서의 핵심어를 추출하거나, 검색 엔진에서 검색 결과의 순위를 결정하거나, 
#문서들 사이의 비슷한 정도를 구하는 등의 용도로 사용할 수 있다.
tfidf = TfidfVectorizer(stop_words='english')
imdb.Description = imdb.Description.fillna('')
tfidf_matrix = tfidf.fit_transform(imdb.Description)
tfidf_matrix.shape

(1000, 5667)

In [19]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix) # 문장들 사이의 유사도를 description으로 구하기

In [23]:
#1.  영화 제목을 인덱스화
indices = pd.Series(imdb.index, index = imdb['Title']).drop_duplicates()
#2. 인덱스를 기준으로 유사도 추출
def rs(title, n=10, sim = cosine_sim):
    idx = indices[title]
    sim_score = list(enumerate(cosine_sim[idx]))
    sim_score = sorted(sim_score, key=lambda x: x[1], reverse=True)
    sim_score = sim_score[1:(n+1)]
    movie_index = [i[0] for i in sim_score]
    return imdb.Title.iloc[movie_index]

In [24]:
rs('The Dark Knight Rises')

54              The Dark Knight
144            Django Unchained
522                      Viking
471            Marie Antoinette
254               The Conjuring
920                   Centurion
253    The Amazing Spider-Man 2
604       The Rise of the Krays
615                Bastille Day
915                     Goksung
Name: Title, dtype: object